## Check List 4 - MotionFilter
* Motion filters are quick decision makers that prevents trying to plan a infeasible motion.
* Here, ReachChecker is tested, compared with actual planning results in random trials
*\* ~~Strikethrough~~ means no-error but none-functional cases.*
* **4-A.3 LatticedChecker**  
  - pick, place
    - single process
    - *multi process is not supported with LatticedChecker, which uses GPU*
    
* You need trained model to use ReachChecker
* To use **LatticedChecker**, get model files from https://github.com/rnb-disinfection/arxiv-highcap/tree/rnb-planning/model/latticized and put them in *$RNB_PLANNING_DIR/model/latticized* folder.
* For robots that are indy7 variants (indy7gripper...), copy indy7 model file and assign the robot name as the file name.

**[Trouble Shooting]**
```
LatticedChecker uses tensorflow model on separate python3 program, through a shared memory.
If some error raises with shared memory, try uncomment and run below cell, to reset shared memory.
```

In [1]:
# import SharedArray as sa
# robot_type_name = "indy7"
# sa.delete("shm://{}.grasp_img".format(robot_type_name))
# sa.delete("shm://{}.arm_img".format(robot_type_name))
# sa.delete("shm://{}.rh_mask".format(robot_type_name))
# sa.delete("shm://{}.result".format(robot_type_name))
# sa.delete("shm://{}.query_in".format(robot_type_name))
# sa.delete("shm://{}.response_out".format(robot_type_name))
# sa.delete("shm://{}.query_quit".format(robot_type_name))

## set running directory

In [2]:
from __future__ import print_function
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

In [3]:
from pkg.controller.combined_robot import *
from pkg.utils.utils import get_now, try_mkdir

In [4]:
VISUALIZE = True
CLEARANCE = 1e-3

# ROBOT_TYPE = RobotType.indy7gripper
# ROBOT_NAME = "indy0"
# TOOL_XYZ = (0,0,0.14)
# TOOL_RPY = (-np.pi/2,0,0)
# GRIP_DEPTH = 0.05
# HOME_POSE = (0,0,0,0,0,0)

ROBOT_TYPE = RobotType.panda
ROBOT_NAME = "panda0"
TOOL_XYZ = (0,0,0.112)
TOOL_RPY = (-np.pi/2,0,0)
GRIP_DEPTH = 0.03
HOME_POSE = (0,-0.3,0,-0.5,0,2.5,0)

## init combined robot config

In [5]:

from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, ROBOT_TYPE, None,
                None)]
              , connection_list=[False])
TOOL_LINK = crob.get_robot_tip_dict().values()[0]
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None)
# # deprecated: s_builder.reset_reference_coord(ref_name="floor")

connection command:
panda0: False


## get ghnd with detected robot config

In [6]:
# xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
xyz_rpy_robots = {ROBOT_NAME: ((0,0,0), (0,0,0))}
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob, start_rviz=VISUALIZE)
HOME_DICT = list2dict(HOME_POSE, gscene.joint_names)


Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0]


In [7]:
from pkg.utils.joint_utils import get_tf
shoulder_link = gscene.urdf_content.joint_map[gscene.joint_names[1]].child
shoulder_height = get_tf(shoulder_link, HOME_DICT, gscene.urdf_content)[2,3]

## add environment

In [8]:
from pkg.geometry.geometry import *
gtems_robot = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=False, collision=True)

## init planning scene

In [9]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

## Register binders

In [10]:
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane, SweepTool

In [11]:
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name=TOOL_LINK, 
                   dims=(0.01,)*3, center=TOOL_XYZ, rpy=TOOL_RPY, color=(1,0,0,1), display=True, collision=False, fixed=True)
gripper = pscene.create_binder(bname="grip0", gname="grip0", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))

Please create a subscriber to the marker


## planner
* Initiailizaing *LatticedChecker* can take ~ 1 min.

In [12]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
from pkg.planning.filtering.latticized_filter import LatticedChecker
mplan = MoveitPlanner(pscene)
gcheck = GraspChecker(pscene)
rcheck = ReachChecker(pscene)
lcheck = LatticedChecker(pscene, gcheck)
checkers_all = [gcheck, rcheck, lcheck]

In [13]:
from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)
tplan.prepare()

# Object Classes

In [14]:
from pkg.utils.gjk import get_point_list, get_gjk_distance
from pkg.planning.constraint.constraint_subject import CustomObject, Grasp2Point, PlacePoint, SweepPoint, SweepTask
from pkg.planning.filtering.lattice_model.scene_building import *

### Test: pick&place motions

##### test few times with visualization

In [15]:
from pkg.planning.constraint.constraint_common import calc_redundancy
import random

Nmax_obj = 3
VISUALIZE = True
result_dict = defaultdict(list)
N_TRIAL = 5
i_count = 0
MP_TIMEOUT = 1

while i_count < N_TRIAL:
    pscene.clear_subjects()
    ## add floor, ceiling
    floor = Floor(gscene, "floor")
    gscene.create_safe(gtype=GEOTYPE.BOX, name="base", link_name="base_link", 
                       dims=(0.2, 0.2, -floor.RTH[2]), center=(0,0,floor.RTH[2]/2), rpy=(0,0,0), 
                       color=floor.geometry.color, display=True, collision=True, fixed=True)
    # ceiling = Ceiling(gscene, "ceiling")

    ## set workplane
    wp = WorkPlane(gscene, "wp", floor_height=floor.RTH[2]+floor.DIM[2]/2)
    pscene.create_binder(bname="wp", gname="wp", _type=PlacePlane, point=None)

    gp_colliding = True
    while gp_colliding:
        ## set goalplane
        gp = SideBox(gscene, "gp", floor_height=wp.RTH[2]+wp.RTH[2]/2)
        gp_colliding = wp.is_overlapped_with(gp.geometry)
    pscene.create_binder(bname="gp", gname="gp", _type=PlacePlane, point=None)


    ## add object
    obj_list = disperse_objects(gscene, PlaneObject, "obj", Nmax_obj, workplane_on=wp, GRIP_DEPTH=GRIP_DEPTH)

    obj_set_list = []
    for i_obj, obj in enumerate(obj_list):
        obj_pscene, handles = add_object(pscene, obj)
        obj_set_list.append((obj, obj_pscene, handles))
    obj, obj_pscene, handles = obj_set_list[0]

    obj_pscene.geometry.color = (0.8,0.2,0.2,1)
    if VISUALIZE:
        gscene.set_rviz()

    initial_state = pscene.initialize_state(HOME_POSE)
    pscene.set_object_state(initial_state)
    tplan.prepare()
    
    scenario = "pick"
    from_state=initial_state
    pscene.set_object_state(from_state)
    gscene.show_pose(from_state.Q)
    
    to_node = ("grip0",)+("wp",)*len(pscene.subject_name_list[1:])
    available_binding_dict = pscene.get_available_binding_dict(from_state, to_node)
    to_state = pscene.sample_leaf_state(from_state, available_binding_dict, to_node)
    obj, handle, actor = to_state.binding_state["obj_0"].get_instance_chain(pscene)
    Q_dict = list2dict(from_state.Q, gscene.joint_names)

    
    for _ in range(10): # try multiple times to get available put position
        to_state = pscene.sample_leaf_state(from_state, available_binding_dict, to_node)
        obj, handle, actor = to_state.binding_state["obj_0"].get_instance_chain(pscene)

        pscene.set_object_state(from_state)
        gscene.show_pose(from_state.Q)
        for mfilter in [gcheck, rcheck]: # test gcheck anc rcheck
            success_mfilter = mfilter.check(to_state.binding_state["obj_0"], Q_dict)
            if not success_mfilter:
                break
        if success_mfilter:
            break
    if not success_mfilter:
        continue
        
    i_count += 1
        
    # test lcheck
    success_mfilter = lcheck.check(to_state.binding_state["obj_0"], Q_dict)

    Traj, LastQ, error, success = mplan.plan_algorithm(
        from_state, to_state, subject_list=pscene.get_changing_subjects(from_state, to_state)[0], timeout=MP_TIMEOUT)

    result_dict[scenario].append((success_mfilter, success))
    if success:
        pscene.set_object_state(from_state)
        if VISUALIZE:
            gscene.show_motion(Traj, period=0.01)
        pick_state = pscene.rebind_all(
            [to_state.binding_state[sname].get_chain() for sname in pscene.subject_name_list], LastQ)
    print("{}/{}: Result {} for {}: {} / {}".format(i_count, N_TRIAL, "match" if success_mfilter==success else "mismatch", scenario, success_mfilter, success))
    # assert success_mfilter == success, "Failure: filter result not same with motion planning result"
    
    if not success:
        pscene.set_object_state(initial_state)
        continue

    scenario = "place"
    from_state=pick_state
    pscene.set_object_state(from_state)
    gscene.show_pose(from_state.Q)
    
    to_node = ("gp",)+("wp",)*len(pscene.subject_name_list[1:])
    available_binding_dict = pscene.get_available_binding_dict(from_state, to_node)
    Q_dict = list2dict(from_state.Q, gscene.joint_names)
    
    for _ in range(10): # try multiple times to get available put position
        to_state = pscene.sample_leaf_state(from_state, available_binding_dict, to_node)
        obj, handle, actor = to_state.binding_state["obj_0"].get_instance_chain(pscene)

        pscene.set_object_state(from_state)
        gscene.show_pose(from_state.Q)
        for mfilter in [gcheck, rcheck]: # test gcheck anc rcheck
            success_mfilter = mfilter.check(to_state.binding_state["obj_0"], Q_dict)
            if not success_mfilter:
                break
        if success_mfilter:
            break
    if not success_mfilter:
        continue
        
    # test lcheck
    success_mfilter = lcheck.check(to_state.binding_state["obj_0"], Q_dict)

    Traj, LastQ, error, success = mplan.plan_algorithm(
        from_state, to_state, subject_list=pscene.get_changing_subjects(from_state, to_state)[0], timeout=MP_TIMEOUT)

    result_dict[scenario].append((success_mfilter, success))
    if success:
        if VISUALIZE:
            gscene.show_motion(Traj, period=0.01)
    pscene.set_object_state(initial_state)
    print("{}/{}: Result {} for {}: {} / {}".format(i_count, N_TRIAL, "match" if success_mfilter==success else "mismatch", scenario, success_mfilter, success))
    # assert success_mfilter == success, "Failure: filter result not same with motion planning result"

1/5: Result match for pick: True / True
2/5: Result match for pick: True / True
3/5: Result mismatch for pick: True / False
4/5: Result match for pick: True / True
5/5: Result mismatch for pick: False / True


##### test many times without visualization

In [16]:
from pkg.planning.constraint.constraint_common import calc_redundancy
import random

Nmax_obj = 3
VISUALIZE = False
result_dict = defaultdict(list)
N_TRIAL = 500
i_count = 0
MP_TIMEOUT = 1

while i_count < N_TRIAL:
    pscene.clear_subjects()
    ## add floor, ceiling
    floor = Floor(gscene, "floor")
    gscene.create_safe(gtype=GEOTYPE.BOX, name="base", link_name="base_link", 
                       dims=(0.2, 0.2, -floor.RTH[2]), center=(0,0,floor.RTH[2]/2), rpy=(0,0,0), 
                       color=floor.geometry.color, display=True, collision=True, fixed=True)
    # ceiling = Ceiling(gscene, "ceiling")

    ## set workplane
    wp = WorkPlane(gscene, "wp", floor_height=floor.RTH[2]+floor.DIM[2]/2)
    pscene.create_binder(bname="wp", gname="wp", _type=PlacePlane, point=None)

    gp_colliding = True
    while gp_colliding:
        ## set goalplane
        gp = SideBox(gscene, "gp", floor_height=wp.RTH[2]+wp.RTH[2]/2)
        gp_colliding = wp.is_overlapped_with(gp.geometry)
    pscene.create_binder(bname="gp", gname="gp", _type=PlacePlane, point=None)


    ## add object
    obj_list = disperse_objects(gscene, PlaneObject, "obj", Nmax_obj, workplane_on=wp, GRIP_DEPTH=GRIP_DEPTH)

    obj_set_list = []
    for i_obj, obj in enumerate(obj_list):
        obj_pscene, handles = add_object(pscene, obj)
        obj_set_list.append((obj, obj_pscene, handles))
    obj, obj_pscene, handles = obj_set_list[0]

    obj_pscene.geometry.color = (0.8,0.2,0.2,1)
    if VISUALIZE:
        gscene.set_rviz()

    initial_state = pscene.initialize_state(HOME_POSE)
    pscene.set_object_state(initial_state)
    tplan.prepare()
    
    scenario = "pick"
    from_state=initial_state
    pscene.set_object_state(from_state)
    gscene.show_pose(from_state.Q)
    
    to_node = ("grip0",)+("wp",)*len(pscene.subject_name_list[1:])
    available_binding_dict = pscene.get_available_binding_dict(from_state, to_node)
    to_state = pscene.sample_leaf_state(from_state, available_binding_dict, to_node)
    obj, handle, actor = to_state.binding_state["obj_0"].get_instance_chain(pscene)
    Q_dict = list2dict(from_state.Q, gscene.joint_names)

    
    for _ in range(10): # try multiple times to get available put position
        to_state = pscene.sample_leaf_state(from_state, available_binding_dict, to_node)
        obj, handle, actor = to_state.binding_state["obj_0"].get_instance_chain(pscene)

        pscene.set_object_state(from_state)
        gscene.show_pose(from_state.Q)
        for mfilter in [gcheck, rcheck]: # test gcheck anc rcheck
            success_mfilter = mfilter.check(to_state.binding_state["obj_0"], Q_dict)
            if not success_mfilter:
                break
        if success_mfilter:
            break
    if not success_mfilter:
        continue
        
    i_count += 1
        
    # test lcheck
    success_mfilter = lcheck.check(to_state.binding_state["obj_0"], Q_dict)

    Traj, LastQ, error, success = mplan.plan_algorithm(
        from_state, to_state, subject_list=pscene.get_changing_subjects(from_state, to_state)[0], timeout=MP_TIMEOUT)

    result_dict[scenario].append((success_mfilter, success))
    if success:
        pscene.set_object_state(from_state)
        if VISUALIZE:
            gscene.show_motion(Traj, period=0.05)
        pick_state = pscene.rebind_all(
            [to_state.binding_state[sname].get_chain() for sname in pscene.subject_name_list], LastQ)
    print("{}/{}: Result {} for {}: {} / {}".format(i_count, N_TRIAL, "match" if success_mfilter==success else "mismatch", scenario, success_mfilter, success))
    # assert success_mfilter == success, "Failure: filter result not same with motion planning result"
    
    if not success:
        pscene.set_object_state(initial_state)
        continue

    scenario = "place"
    from_state=pick_state
    pscene.set_object_state(from_state)
    gscene.show_pose(from_state.Q)
    
    to_node = ("gp",)+("wp",)*len(pscene.subject_name_list[1:])
    available_binding_dict = pscene.get_available_binding_dict(from_state, to_node)
    Q_dict = list2dict(from_state.Q, gscene.joint_names)
    
    for _ in range(10): # try multiple times to get available put position
        to_state = pscene.sample_leaf_state(from_state, available_binding_dict, to_node)
        obj, handle, actor = to_state.binding_state["obj_0"].get_instance_chain(pscene)

        pscene.set_object_state(from_state)
        gscene.show_pose(from_state.Q)
        for mfilter in [gcheck, rcheck]: # test gcheck anc rcheck
            success_mfilter = mfilter.check(to_state.binding_state["obj_0"], Q_dict)
            if not success_mfilter:
                break
        if success_mfilter:
            break
    if not success_mfilter:
        continue
        
    # test lcheck
    success_mfilter = lcheck.check(to_state.binding_state["obj_0"], Q_dict)

    Traj, LastQ, error, success = mplan.plan_algorithm(
        from_state, to_state, subject_list=pscene.get_changing_subjects(from_state, to_state)[0], timeout=MP_TIMEOUT)

    result_dict[scenario].append((success_mfilter, success))
    if success:
        if VISUALIZE:
            gscene.show_motion(Traj, period=0.05)
    pscene.set_object_state(initial_state)
    print("{}/{}: Result {} for {}: {} / {}".format(i_count, N_TRIAL, "match" if success_mfilter==success else "mismatch", scenario, success_mfilter, success))
    # assert success_mfilter == success, "Failure: filter result not same with motion planning result"

1/500: Result match for pick: True / True
1/500: Result mismatch for place: False / True
2/500: Result match for pick: False / False
3/500: Result match for pick: True / True
3/500: Result match for place: False / False
4/500: Result match for pick: True / True
5/500: Result match for pick: True / True
6/500: Result match for pick: True / True
7/500: Result match for pick: True / True
8/500: Result match for pick: True / True
9/500: Result match for pick: True / True
10/500: Result match for pick: True / True
11/500: Result match for pick: True / True
12/500: Result match for pick: True / True
12/500: Result match for place: False / False
13/500: Result mismatch for pick: True / False
14/500: Result match for pick: True / True
15/500: Result match for pick: True / True
16/500: Result mismatch for pick: True / False
17/500: Result match for pick: True / True
17/500: Result mismatch for place: False / True
18/500: Result match for pick: False / False
19/500: Result match for pick: True /

146/500: Result match for pick: True / True
146/500: Result match for place: True / True
147/500: Result match for pick: True / True
147/500: Result mismatch for place: True / False
148/500: Result match for pick: True / True
149/500: Result match for pick: True / True
150/500: Result match for pick: True / True
150/500: Result mismatch for place: True / False
151/500: Result match for pick: True / True
151/500: Result match for place: False / False
152/500: Result match for pick: True / True
152/500: Result mismatch for place: False / True
153/500: Result match for pick: True / True
154/500: Result match for pick: True / True
154/500: Result match for place: True / True
155/500: Result match for pick: True / True
155/500: Result match for place: True / True
156/500: Result match for pick: True / True
157/500: Result match for pick: True / True
158/500: Result match for pick: True / True
159/500: Result match for pick: True / True
160/500: Result match for pick: True / True
160/500: Re

292/500: Result match for pick: True / True
293/500: Result match for pick: True / True
294/500: Result match for pick: True / True
295/500: Result match for pick: True / True
296/500: Result mismatch for pick: True / False
297/500: Result match for pick: True / True
298/500: Result match for pick: True / True
298/500: Result match for place: False / False
299/500: Result match for pick: True / True
299/500: Result match for place: True / True
300/500: Result match for pick: True / True
301/500: Result match for pick: True / True
302/500: Result match for pick: True / True
302/500: Result match for place: True / True
303/500: Result match for pick: True / True
303/500: Result match for place: True / True
304/500: Result match for pick: True / True
305/500: Result match for pick: True / True
305/500: Result match for place: True / True
306/500: Result mismatch for pick: True / False
307/500: Result match for pick: True / True
307/500: Result mismatch for place: True / False
308/500: Res

433/500: Result mismatch for pick: True / False
434/500: Result match for pick: True / True
434/500: Result match for place: True / True
435/500: Result match for pick: True / True
436/500: Result match for pick: True / True
437/500: Result match for pick: True / True
437/500: Result match for place: False / False
438/500: Result match for pick: True / True
438/500: Result match for place: False / False
439/500: Result match for pick: True / True
440/500: Result match for pick: True / True
441/500: Result match for pick: True / True
442/500: Result match for pick: True / True
443/500: Result match for pick: True / True
443/500: Result match for place: False / False
444/500: Result match for pick: True / True
445/500: Result match for pick: True / True
445/500: Result match for place: True / True
446/500: Result match for pick: True / True
447/500: Result match for pick: True / True
447/500: Result match for place: True / True
448/500: Result match for pick: True / True
449/500: Result 

In [17]:
ACCURACY_CUT = 80
print("Accuracy cutline: {} %".format(ACCURACY_CUT))
for k, v in sorted(result_dict.items()):
    v_arr = np.array(v)
    suc_idx = np.where(v_arr[:, 1])[0]
    fail_idx = np.where(np.logical_not(v_arr[:, 1]))[0]
    acc_val = np.round(np.mean(v_arr[:,0]==v_arr[:,1])*100, 1)
    acc_suc = np.round(np.mean(v_arr[suc_idx,0]==v_arr[suc_idx,1])*100, 1)
    acc_fail = np.round(np.mean(v_arr[fail_idx,0]==v_arr[fail_idx,1])*100, 1)
    print("="*60)
    print("{} accuracy: {} % / success accuracy: {} % / fail accuracy: {} % / success count:{}/{}".format(k, acc_val, acc_suc, acc_fail, len(suc_idx), len(v_arr)))
    assert acc_val>ACCURACY_CUT and acc_suc>ACCURACY_CUT, \
            "Accuracy {}/{} is lower than {}. Consider debugging the algorithm.".format(acc_val, acc_suc, ACCURACY_CUT)
    print("""OK with "{}" scenario""".format(k))

Accuracy cutline: 80 %
pick accuracy: 83.8 % / success accuracy: 98.1 % / fail accuracy: 16.1 % / success count:413/500
OK with "pick" scenario
place accuracy: 83.5 % / success accuracy: 90.6 % / fail accuracy: 72.2 % / success count:85/139
OK with "place" scenario


### planning: pick&place

In [18]:
from pkg.planning.constraint.constraint_common import calc_redundancy
from copy import deepcopy

pscene.clear_subjects()
    
Nmax_obj = 1
gtimer = GlobalTimer.instance()
gtimer.reset()
## add floor, ceiling
floor = Floor(gscene, "floor")
gscene.create_safe(gtype=GEOTYPE.BOX, name="base", link_name="base_link", 
                   dims=(0.2, 0.2, -floor.RTH[2]), center=(0,0,floor.RTH[2]/2), rpy=(0,0,0), 
                   color=floor.geometry.color, display=True, collision=True, fixed=True)
# ceiling = Ceiling(gscene, "ceiling")

## set workplane
wp = WorkPlane(gscene, "wp", floor_height=floor.RTH[2]+floor.DIM[2]/2)
pscene.create_binder(bname="wp", gname="wp", _type=PlacePlane, point=None)

gp_colliding = True
while gp_colliding:
    ## set goalplane
    gp = WorkPlane(gscene, "gp", floor_height=wp.RTH[2]+wp.RTH[2]/2)
    gp_colliding = wp.is_overlapped_with(gp.geometry)
pscene.create_binder(bname="gp", gname="gp", _type=PlacePlane, point=None)


## add object
obj_list = disperse_objects(gscene, PlaneObject, "obj", Nmax_obj, workplane_on=wp, GRIP_DEPTH=GRIP_DEPTH)

obj_set_list = []
for i_obj, obj in enumerate(obj_list):
    obj_pscene, handles = add_object(pscene, obj)
    obj_set_list.append((obj, obj_pscene, handles))
obj, obj_pscene, handles = obj_set_list[0]

obj_pscene.geometry.color = (0.8,0.2,0.2,1)
if VISUALIZE:
    gscene.set_rviz()

mplan.update_gscene()
initial_state = pscene.initialize_state(HOME_POSE)
pscene.set_object_state(initial_state)
from_state = initial_state.copy(pscene)

ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

gtimer = GlobalTimer.instance()
gtimer.reset()

mplan.motion_filters = checkers_all
goal_nodes = [("gp",)+deepcopy(from_state.node)[1:]]
mplan.reset_log(True)
gtimer.tic("plan")
ppline.search(initial_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, 
              timeout_loop=10, multiprocess=True, timeout=1, add_homing=False)
elapsed = gtimer.toc("plan")/1000
schedule = tplan.get_best_schedule(at_home=False)
res = len(schedule)>0
if res:
    ppline.play_schedule(schedule)
else:
    raise(RuntimeError("Failed to make plan - may be impossible problem was generated. try again up to 10 times or debug"))

Use 18/36 agents
try: 0 - ('wp',)->('grip0',)
Motion Filer Failure: ReachChecker
result: 0 - ('wp',)->('grip0',) = fail
try: 0 - ('wp',)->('grip0',)
Motion Filer Failure: ReachChecker
result: 0 - ('wp',)->('grip0',) = fail
try: 0 - ('wp',)->('grip0',)
try: 0 - ('wp',)->('grip0',)
Motion Filer Failure: GraspChecker
result: 0 - ('wp',)->('grip0',) = fail
Motion Filer Failure: GraspChecker
result: 0 - ('wp',)->('grip0',) = fail
try: 0 - ('wp',)->('grip0',)
try: 0 - ('wp',)->('grip0',)
Motion Filer Failure: GraspChecker
Motion Filer Failure: GraspChecker
result: 0 - ('wp',)->('grip0',) = fail
result: 0 - ('wp',)->('grip0',) = fail
try: 0 - ('wp',)->('grip0',)
try: 0 - ('wp',)->('grip0',)
Motion Filer Failure: GraspChecker
Motion Filer Failure: ReachChecker
result: 0 - ('wp',)->('grip0',) = fail
result: 0 - ('wp',)->('grip0',) = fail
try: 0 - ('wp',)->('grip0',)
try: 0 - ('wp',)->('grip0',)
Motion Filer Failure: GraspChecker
Motion Filer Failure: GraspChecker
result: 0 - ('wp',)->('grip0',)

result: 2 - ('grip0',)->('gp',) = fail
result: 0 - ('wp',)->('grip0',) = fail
Motion Filer Failure: ReachChecker
try: 1 - ('grip0',)->('wp',)
result: 0 - ('wp',)->('grip0',) = fail
Motion Filer Failure: ReachChecker
Motion Filer Failure: ReachChecker
try: 0 - ('wp',)->('grip0',)
result: 1 - ('grip0',)->('gp',) = fail
Goal reached
result: 1 - ('grip0',)->('gp',) = fail
result: 1 - ('grip0',)->('gp',) = fail
try: 0 - ('wp',)->('grip0',)
error: 1.1
try: 1 - ('grip0',)->('gp',)
Motion Filer Failure: GraspChecker
try: 2 - ('grip0',)->('gp',)
result: 1 - ('grip0',)->('wp',) = fail
result: 0 - ('wp',)->('grip0',) = fail
try: 2 - ('grip0',)->('wp',)
result: 1 - ('grip0',)->('gp',) = success
try: 0 - ('wp',)->('grip0',)
try: 2 - ('grip0',)->('gp',)
result: 2 - ('grip0',)->('gp',) = fail
result: 0 - ('wp',)->('grip0',) = fail
try transition motion
Motion Filer Failure: ReachChecker
Motion Filer Failure: ReachChecker
result: 2 - ('grip0',)->('wp',) = fail
branching: 1->4 (0.9/10.0 s, steps/err: 6

Goal reached
result: 1 - ('grip0',)->('gp',) = success
branching: 1->12 (1.15/10.0 s, steps/err: 77(253.820896149 ms)/0.00161469334322)
======================= terminated 3: required answers acquired  (1.2/10.0) ===============================
error: 2.4
try transition motion
error: 1.1
try transition motion
transition motion tried: True
Goal reached
result: 2 - ('grip0',)->('gp',) = success
branching: 2->13 (1.19/10.0 s, steps/err: 61(271.522045135 ms)/0.00179390967718)
======================= terminated 15: required answers acquired  (1.2/10.0) ===============================
error: 0.99
try transition motion
transition motion tried: True
Goal reached
result: 2 - ('grip0',)->('gp',) = success
branching: 2->14 (1.23/10.0 s, steps/err: 89(305.637121201 ms)/0.00135089347277)
======================= terminated 8: required answers acquired  (1.2/10.0) ===============================
transition motion tried: True
transition motion tried: True
Goal reached
transition motion tried: True
resu

In [19]:
ppline.play_schedule(schedule)

('wp',)->('grip0',)
('grip0',)->('gp',)
